In [2]:
import pandas as pd
import json
import ast
import os
from tqdm import tqdm
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD

In [5]:
data = pd.read_csv(r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\dataframes\playlists_uri.csv')
data.drop('name', axis=1, inplace=True)

data.head()


,user_id,tracks
0,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,1,"[52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 6..."
2,2,"[91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101,..."
3,3,"[155, 156, 157, 158, 159, 160, 161, 162, 163, ..."
4,4,"[281, 282, 283, 284, 285, 286, 287, 288, 289, ..."


In [6]:
# Step 1: Extract rows, columns, and values for the sparse matrix
rows = []
cols = []
values = []

for index, row in tqdm(data.iterrows()):
    try:
        user_id = row["user_id"]  # Extract user_id
        tracks = row["tracks"]    # Extract tracks
        rows.extend([user_id] * len(tracks))  # Repeat user_id for each track
        cols.extend(tracks)  # Track indices as columns
        values.extend([1] * len(tracks))  # All values are 1
    except Exception as e:
        print(user_id)
        print(e)


1000000it [04:06, 4057.61it/s]


In [7]:
# Save the DataFrame to a file
salva = True
if salva:
    output_dir = r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System'
    os.makedirs(output_dir, exist_ok=True)

    # Save as CSV instead of Parquet
    output_file = os.path.join(output_dir, "playlists_trackId.csv")
    data.to_csv(output_file, index=False)


    print(f"Processing completed. DataFrame saved to {output_file}.")

Processing completed. DataFrame saved to C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\playlists_trackId.csv.


In [8]:
# Step 2: Create the sparse matrix
sparse_matrix = coo_matrix((values, (rows, cols)), shape=(1000001, 2262293))

MemoryError: Unable to allocate 3.38 GiB for an array with shape (453086881,) and data type int64

In [20]:
# Step 3: Apply Truncated SVD
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD

# Convert COO to CSR for better compatibility
csr = sparse_matrix.tocsr()

from scipy.sparse.linalg import svds

# Perform Truncated SVD with a limited number of components
k = 5  # Number of components
U, S, Vt = svds(csr, k=k)

print("U Matrix:", U)
print("Singular Values:", S)
print("Vt Matrix:", Vt)


MemoryError: Unable to allocate 506. MiB for an array with shape (66346428,) and data type int64

In [7]:
# Create a matrix where rows are users and columns are movies, and the values are ratings
utility_matrix = data.pivot(index='user_id', columns='movieId', values='rating').fillna(0)
print("Utility Matrix Shape:", utility_matrix.shape)

# Convert to numpy array
R = utility_matrix.to_numpy()

KeyError: 'userId'